# Facial Emotion Recognition using CNN

In [1]:
import numpy as np
import cv2

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

2023-07-16 19:30:50.579478: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 19:30:50.618058: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-16 19:30:50.618693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 19:30:51.353115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir = "../dataset/train"
val_dir = "../dataset/test"
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
)
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
# Build the CNN model
# Layers in Sequential are stacked on top of each other
emotion_model = Sequential()

# 2D Conv layer with 32 filters of size 3x3. Input shape is (48,48,1), for
# grayscale images of size 48x48.
emotion_model.add(
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(48, 48, 1))
)
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation="relu"))
emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7, activation="softmax"))

2023-07-16 19:30:56.512311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-16 19:30:56.535994: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# Train the model
emotion_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0001, weight_decay=1e-6),
    metrics=["accuracy"],
)
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64,
)

Epoch 1/50


/tmp/ipykernel_1347621/2683148763.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 81s 179ms/step - loss: 1.7746 - accuracy: 0.2789 - val_loss: 1.6423 - val_accuracy: 0.3682
Epoch 2/50
448/448 [==============================] - 75s 168ms/step - loss: 1.5987 - accuracy: 0.3810 - val_loss: 1.5414 - val_accuracy: 0.4196
Epoch 3/50
448/448 [==============================] - 75s 168ms/step - loss: 1.5123 - accuracy: 0.4186 - val_loss: 1.4616 - val_accuracy: 0.4502
Epoch 4/50
448/448 [==============================] - 83s 184ms/step - loss: 1.4449 - accuracy: 0.4449 - val_loss: 1.4000 - val_accuracy: 0.4623
Epoch 5/50
448/448 [==============================] - 76s 170ms/step - loss: 1.3893 - accuracy: 0.4721 - val_loss: 1.3616 - val_accuracy: 0.4840
Epoch 6/50
448/448 [==============================] - 77s 173ms/step - loss: 1.3384 - accuracy: 0.4928 - val_loss: 1.3082 - val_accuracy: 0.5028
Epoch 7/50
448/448 [==============================] - 75s 167ms/step - loss: 1.2912 - accuracy: 0.5130 - val_loss: 1.2859 - val_accuracy: 0.5

In [6]:
# save the model weigh
emotion_model.save_weights('model.h5')

In [9]:
# using OpenCV haarcascade xml detect the bounding boxes of face in the webcam and predict the emotions
cv2.ocl.setUseOpenCL(False)

emotion_dict = {
    0: "Angry",
    1: "Disgusted",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised",
}

cap = cv2.VideoCapture(0)

# load the cascade classifier before loop
haarcascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
bounding_box = cv2.CascadeClassifier(haarcascade_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(
        gray_frame, scaleFactor=1.3, minNeighbors=5
    )

    for (x,y,w,h) in num_faces:
        cv2.rectangle(frame,(x,y-50),(x+w,y+h+10),(255,0,0),2)
        roi_gray_frame=gray_frame[y:y+h,x:x+w]
        cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0
        emotion_prediction=emotion_model.predict(cropped_img)
        maxindex=int(np.argmax(emotion_prediction))
        cv2.putText(frame,emotion_dict[maxindex],(x+20,y-60),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
    cv2.imshow('Video',cv2.resize(frame,(1200,860),interpolation=cv2.INTER_CUBIC))
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread (0x9bb5ce0).
Cannot move to target thread (0x7598980)

QObject::moveToThread: Current thread (0x7598980) is not the object's thread

1/1 [==============================] - 0s 19ms/step


KeyboardInterrupt: 

### Emotion detection result
![](../docs/figures/surprised.png)